In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_C_058, input_features_C_058 = pre_models.Input_C_058('weights/')
    
    outputs = ['Input_C_058']

    preprocessings = ['min_max_normalization']

    models = [model_C_058]

    input_features = [input_features_C_058]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=9
data=pd.read_csv('./BayesianRidge_Pre_0/Input_C_058.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_054', 'Input_C_052', 'Input_C_055', 'Input_C_135',
       'Input_C_059', 'Input_C_060', 'Input_C_056', 'Input_C_053',
       'Input_A4_011'], dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [11]:
nn_ave(nn, y)

test_rmse_ave: 0.003060707047082828
[0.00383050913293738, 0.003336432716468487, 0.0021065044439281667, 0.003571376173347496, 0.0031061971794034806, 0.001920100733095955, 0.002785437032873378, 0.0025469611057583165, 0.0038295722890587984, 0.003573979663956822]


test_r2_ave: 0.6464219766975188
[0.3910234177211709, 0.587235346451389, 0.851122174423196, 0.6290778121470554, 0.6503971709628651, 0.8903261557756506, 0.7271269491047998, 0.7934870314154366, 0.45177755895810967, 0.49264615001551315]


## Linear Regression Baseline

In [12]:
lr_rmse_ave_fea(skew_data,9)

train_rmse: 0.0018062772782699791
test_rmse: 0.0018484704121483684
test_r2: 0.8802204885524739


## Lasso

In [13]:
laso_rmse_ave(skew_data[title],y,5e-06)

test_rmse_ave: 0.001788355218182537
[0.0018684381774833917, 0.0017178219841128728, 0.0019609645205860083, 0.0018380530704643044, 0.0018853680084753317, 0.001735840238042914, 0.0015637775092538161, 0.0018964500318850819, 0.0016643424606024285, 0.0017524961809192224]


test_r2_ave: 0.8873953987308474
[0.8551077844283718, 0.8905806651122459, 0.8709836562623341, 0.9017510964422667, 0.8712019868919126, 0.910365635785961, 0.9139949641287496, 0.8855052660467183, 0.8964520152040906, 0.8780109170058238]


## ElasticNet

In [14]:
ElasticNet_rmse_ave(skew_data[title],y,[0.007],0.01)

test_rmse_ave: 0.0017920703220896312
[0.0018500499511447746, 0.0017193800735374853, 0.0019760453490751738, 0.0018906539812752275, 0.0018966281677105343, 0.0017317409901115518, 0.0015563086417196273, 0.0019231104685331066, 0.0016299965300786575, 0.001746789067710173]


test_r2_ave: 0.8870377907260281
[0.8579456631279614, 0.8903820852519976, 0.868991621273978, 0.8960473114344246, 0.86965892795612, 0.9107884853322245, 0.914814551577135, 0.8822634874151826, 0.9006816204765453, 0.8788041534147102]


## SVR

In [15]:
model=SVR(kernel='linear', C=1.25, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,9,model)

train_rmse: 0.0018584341791834512
test_rmse: 0.0018916388734179254
test_r2: 0.8746508093208286


## XGB

In [16]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'dart', 
              n_estimators = 125, 
              max_depth = 5, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.5,
              reg_alpha =  0,
              reg_lambda = 2)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.001955727904184468
[0.0018466467985351538, 0.001998024888860786, 0.0023356968090925893, 0.00195638810302729, 0.0020305038666861273, 0.0020266491759792387, 0.0017500617708297594, 0.0019470171555288952, 0.0017123616239316656, 0.001953928849373172]


test_r2_ave: 0.8654872359887866
[0.8584677981672647, 0.8519734745131992, 0.8169632934941589, 0.8886932132956071, 0.8506089643358404, 0.8778166038434239, 0.8922838774074072, 0.8793180663667776, 0.8903907420354701, 0.8483563264287175]


## RandomForest

In [17]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 60,
            max_depth = 8,
            min_samples_split = 3,
            max_leaf_nodes = 12,
            min_samples_leaf =4,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.0019141087235315817
[0.0018577097224646156, 0.0018288595900760132, 0.0022601966593258542, 0.0019292934406905775, 0.002022019095598604, 0.0020978843597729575, 0.001686142272855897, 0.0018281012695572562, 0.0017198735048260015, 0.001911007320148044]


test_r2_ave: 0.8712026715284589
[0.856766931412636, 0.8759780699202568, 0.8286051709686606, 0.8917549128190362, 0.8518548623985964, 0.8690763423173369, 0.9000086598305587, 0.893609413213082, 0.8894269529794431, 0.8549453994249814]


In [8]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=9e-05)
elastic_mod=ElasticNet(alpha=[0.006], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.25, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'dart', 
              n_estimators = 125, 
              max_depth = 5, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.5,
              reg_alpha =  0,
              reg_lambda = 2)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 60,
            max_depth = 8,
            min_samples_split = 3,
            max_leaf_nodes = 12,
            min_samples_leaf =4,
            random_state = 42)  
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('RandomForest', random_mod),('XGB', xgb_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.001787723489802094
[0.0018232517520947711, 0.0017356541130508739, 0.0020026993055085335, 0.0018643656703483309, 0.0019031865437230723, 0.0017742286780573191, 0.0015548541849047396, 0.00186600600663363, 0.0016166513154695233, 0.001736337328230149]


test_r2_ave: 0.8876470005024135
[0.862031206233649, 0.8882971840829769, 0.8654335633868929, 0.8989180028855511, 0.8687559531594173, 0.906357234922262, 0.9149736982369029, 0.889151765992241, 0.9023012549498184, 0.8802501411744242]


In [9]:

stack_mod = StackingRegressor(regressors=[lasso_mod, lr, elastic_mod], 
                           meta_regressor=vote_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.001801823313093958
[0.001889870350550944, 0.0017223092689039672, 0.002003225698042372, 0.001851313962072251, 0.0019407150537209985, 0.0018107959320145701, 0.0015342279736196118, 0.0018921089600944258, 0.0016283444531392803, 0.0017453214787811588]


test_r2_ave: 0.8856584549573592
[0.8517647086171314, 0.8900082693576837, 0.8653628147962257, 0.9003283215030163, 0.8635289774776655, 0.9024574601207866, 0.9172146003041513, 0.8860288348072163, 0.9008828459997777, 0.8790077165899375]


In [10]:
ave(skew_data[title],y,0.1,stack_mod,0.45,vote_mod,0.45,lasso_mod)

test_rmse_ave: 0.001784266339980882
[0.001835458992628435, 0.0017230647094753, 0.001979987837231587, 0.0018739100929601922, 0.001897391722393911, 0.0017514902339073725, 0.0015495695558029217, 0.0018806388677378326, 0.001615482955020703, 0.0017356684326505637]


test_r2_ave: 0.8880476029520297
[0.8601775330548431, 0.8899117588873244, 0.8684683395327114, 0.8978803965496025, 0.8695539600231885, 0.9087420981103579, 0.9155506898015382, 0.8874064483269962, 0.9024424184785872, 0.8803423867551478]


In [12]:
name='Input_C_058'
test_pred=[]
seed=[0,1,5,6,8,9,3,4]
stack_w=0.1
vote_w=0.45
model_w=0.45
model=lasso_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)


final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))
